In [27]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint


In [2]:
def extract_frames_from_video(video_path, frame_count=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(total_frames // frame_count, 1)

    for i in range(0, total_frames, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    cap.release()
    return frames

In [3]:
def load_video_dataset(data_dir, frame_count=5):
    X, y = [], []
    class_labels = {'YouTube-real': 0, 'Celeb-synthesis': 1, 'Celeb-real': 0}
    
    for label, value in class_labels.items():
        subdir = os.path.join(data_dir, label)
        for file in os.listdir(subdir):
            if file.endswith('.mp4'):
                video_path = os.path.join(subdir, file)
                frames = extract_frames_from_video(video_path, frame_count=frame_count)
                for frame in frames:
                    X.append(frame)
                    y.append(value)
    
    return X, y


In [4]:
def apply_sobel_filter(image, target_size=(256, 256)):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        sobel = cv2.magnitude(sobelx, sobely)
        sobel_resized = cv2.resize(sobel, target_size)
        return sobel_resized
    except Exception as e:
        print(f"Erro ao aplicar o filtro Sobel: {e}")
        return None

In [5]:
def process_in_batches(X, batch_size=100, target_size=(256, 256)):
    X_processed = []
    for i in range(0, len(X), batch_size):
        batch = X[i:i + batch_size]
        batch_sobel = [apply_sobel_filter(frame, target_size) for frame in batch]
        batch_sobel = [sobel for sobel in batch_sobel if sobel is not None]
        X_processed.extend(batch_sobel)
    return np.array(X_processed)


In [6]:
# Carregar o conjunto de dados de vídeos
data_dir = 'Celeb-DF'
X, y = load_video_dataset(data_dir)

In [7]:
# Processar em lotes
X_sobel = process_in_batches(X, batch_size=100, target_size=(256, 256))

# Normalizar os dados
X_sobel = X_sobel / np.max(X_sobel)

# Expandir as dimensões para incluir o canal de cor
X_sobel = np.expand_dims(X_sobel, axis=-1)

In [10]:
# Achatar as imagens de (256, 256, 1) para (256*256,)
n_samples = X_sobel.shape[0]
n_features = X_sobel.shape[1] * X_sobel.shape[2]
X_sobel_flattened = X_sobel.reshape(n_samples, n_features)

# Aplicar LDA para reduzir a dimensionalidade e otimizar a separação entre classes
lda = LDA(n_components=1)  # Como existem duas classes, usamos 1 componente
X_lda = lda.fit_transform(X_sobel_flattened, y)

# Expandir as dimensões novamente para usar como entrada da CNN
X_lda = np.expand_dims(X_lda, axis=-1)

In [11]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_sobel, y, test_size=0.2, random_state=42)

In [12]:
# Adicionar um canal de cor extra para compatibilidade com EfficientNet
X_train = np.repeat(X_train, 3, axis=-1)
X_test = np.repeat(X_test, 3, axis=-1)

In [13]:
# Pré-processar os dados com preprocess_input do EfficientNet
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [23]:
# Construir e treinar o modelo EfficientNetB0
def build_efficientnet(input_shape):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [15]:
input_shape = (X_sobel.shape[1], X_sobel.shape[2], 3)
efficientnet_model = build_efficientnet(input_shape)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [30]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', 
                                                 monitor='val_accuracy', 
                                                 save_best_only=True, 
                                                 mode='max', 
                                                 verbose=1)

efficientnet_model.fit(X_train, np.array(y_train), epochs=10, batch_size=32, validation_data=(X_test, np.array(y_test)), callbacks=[checkpoint])

Epoch 1/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9308 - loss: 0.1842
Epoch 1: val_accuracy improved from -inf to 0.37779, saving model to best_model.keras
174/174 ━━━━━━━━━━━━━━━━━━━━ 516s 3s/step - accuracy: 0.9308 - loss: 0.1842 - val_accuracy: 0.3778 - val_loss: 2.9469
Epoch 2/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9347 - loss: 0.1756
Epoch 2: val_accuracy improved from 0.37779 to 0.55660, saving model to best_model.keras
174/174 ━━━━━━━━━━━━━━━━━━━━ 501s 3s/step - accuracy: 0.9346 - loss: 0.1756 - val_accuracy: 0.5566 - val_loss: 2.0240
Epoch 3/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9453 - loss: 0.1470
Epoch 3: val_accuracy improved from 0.55660 to 0.75126, saving model to best_model.keras
174/174 ━━━━━━━━━━━━━━━━━━━━ 502s 3s/step - accuracy: 0.9453 - loss: 0.1470 - val_accuracy: 0.7513 - val_loss: 0.7075
Epoch 4/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9636 - loss: 0.1146
Epoch 4: val_accuracy improved from 0.7512

In [33]:
# Load the best model and evaluate
best_model = tf.keras.models.load_model('best_model.keras')
test_loss, test_accuracy = best_model.evaluate(X_test, np.array(y_test))
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Predictions and evaluation
y_pred = (best_model.predict(X_test) > 0.5).astype(int)
print("Classification Report:")
print(classification_report(y_test, y_pred))

c:\Users\Luca\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 217 variables whereas the saved optimizer has 432 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


44/44 ━━━━━━━━━━━━━━━━━━━━ 25s 508ms/step - accuracy: 0.3257 - loss: 15229.0566
Test Loss: 14849.4091796875
Test Accuracy: 0.34246575832366943
44/44 ━━━━━━━━━━━━━━━━━━━━ 23s 495ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.34      1.00      0.51       475
           1       0.00      0.00      0.00       912

    accuracy                           0.34      1387
   macro avg       0.17      0.50      0.26      1387
weighted avg       0.12      0.34      0.17      1387



c:\Users\Luca\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Luca\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Luca\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [17]:
efficientnet_model.save('Model/meu_modelo_Sobel_Efficientnet_Model_LDA.h5')

In [18]:
def evaluate_model(model_path, X_test, y_test, class_names=['Class 0', 'Class 1']):
    """
    Avalia um modelo carregado em termos de acurácia, ROC-AUC e relatório de classificação.
    
    :param model_path: Caminho para o arquivo do modelo.
    :param X_test: Dados de teste.
    :param y_test: Rótulos de teste.
    :param class_names: Nomes das classes para o relatório de classificação.
    :return: Tuple contendo a acurácia, o ROC-AUC e o relatório de classificação.
    """
    # Carregar o modelo
    model = load_model(model_path)
    
    # Fazer previsões
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)
    
    # Calcular a acurácia
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calcular o ROC-AUC
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    
    # Gerar o relatório de classificação
    report = classification_report(y_test, y_pred, target_names=class_names)
    
    return accuracy, roc_auc, report

In [22]:
# Usar a função para comparar os modelos
model_paths = ['Model/meu_modelo_Sobel_Efficientnet_Model_LDA.h5']
for path in model_paths:
    accuracy, roc_auc, report = evaluate_model(path, X_test, y_test)
    print(f"Modelo: {path}")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print("Relatório de Classificação:")
    print(report)

44/44 ━━━━━━━━━━━━━━━━━━━━ 23s 502ms/step
Modelo: Model/meu_modelo_Sobel_Efficientnet_Model_LDA.h5
Acurácia: 0.4780
ROC-AUC: 0.9126
Relatório de Classificação:
              precision    recall  f1-score   support

     Class 0       0.39      0.97      0.56       475
     Class 1       0.94      0.22      0.36       912

    accuracy                           0.48      1387
   macro avg       0.67      0.60      0.46      1387
weighted avg       0.75      0.48      0.43      1387

